# Constrained runs

Conduct future climate projections by constrained runs

In [1]:
import sys
import yaml
import numpy as np
import pandas as pd
from netCDF4 import Dataset
from scipy.interpolate import interp1d
from tqdm.notebook import tqdm

In [2]:
# mce is available in https://github.com/tsutsui1872/mce 
sys.path.append('../mce')

In [3]:
from mce.core.forcing import RfAll

In [4]:
from src.util import RetrieveGitHub, retrieve_url, dffilter, df2nc
from src.tlm import DriverMCE

In [5]:
owner = 'IPCC-WG1'
repo = 'Chapter-7'
repo_ch7 = RetrieveGitHub(owner, repo, './datain')

## Forcing input

In [6]:
scenarios = [
    'ssp119', 'ssp126', 'ssp245', 'ssp370', 'ssp585',
]

In [7]:
df = {}
for scenario in scenarios:
    path = repo_ch7.retrieve(f'data_output/SSPs/ERF_{scenario}_1750-2500.csv')
    df[scenario] = pd.read_csv(path, index_col=0).T

[2024-06-24 11:23:58 mce.util] INFO:Use local file datain/IPCC-WG1/Chapter-7/data_output/SSPs/ERF_ssp119_1750-2500.csv retrieved from https://github.com/IPCC-WG1/Chapter-7/raw/main/data_output/SSPs/ERF_ssp119_1750-2500.csv on 2024-06-21
[2024-06-24 11:23:58 mce.util] INFO:Use local file datain/IPCC-WG1/Chapter-7/data_output/SSPs/ERF_ssp126_1750-2500.csv retrieved from https://github.com/IPCC-WG1/Chapter-7/raw/main/data_output/SSPs/ERF_ssp126_1750-2500.csv on 2024-06-21
[2024-06-24 11:23:58 mce.util] INFO:Use local file datain/IPCC-WG1/Chapter-7/data_output/SSPs/ERF_ssp245_1750-2500.csv retrieved from https://github.com/IPCC-WG1/Chapter-7/raw/main/data_output/SSPs/ERF_ssp245_1750-2500.csv on 2024-06-21
[2024-06-24 11:23:58 mce.util] INFO:Use local file datain/IPCC-WG1/Chapter-7/data_output/SSPs/ERF_ssp370_1750-2500.csv retrieved from https://github.com/IPCC-WG1/Chapter-7/raw/main/data_output/SSPs/ERF_ssp370_1750-2500.csv on 2024-06-21
[2024-06-24 11:23:58 mce.util] INFO:Use local file d

In [8]:
df_erf = pd.concat(df)
df_erf

year                            1750          1751          1752  \
ssp119 co2                  0.000000  1.416069e-03  2.831801e-03   
       ch4                  0.000000  5.082820e-04  1.016257e-03   
       n2o                  0.000000  7.397683e-05  1.479456e-04   
       other_wmghg          0.000000  7.958010e-10  1.591602e-09   
       o3                   0.000000  3.000000e-04  6.000000e-04   
...                              ...           ...           ...   
ssp585 volcanic             0.199978  1.999904e-01  1.999945e-01   
       solar                0.097590  7.924397e-02  4.916604e-02   
       total_anthropogenic  0.000000  7.031869e-03  1.227668e-02   
       total_natural        0.297568  2.792343e-01  2.491606e-01   
       total                0.297568  2.862662e-01  2.614372e-01   

year                                1753          1754          1755  \
ssp119 co2                  4.247196e-03  5.662254e-03  7.076975e-03   
       ch4                  1.523925e-03  2.031287e-03  2.538343e-03   
       n2o                  2.219062e-04  2.958588e-04  3.698032e-04   
       other_wmghg          2.387403e-09  3.183204e-09  3.979005e-09   
       o3                   9.000000e-04  1.200000e-03  1.500000e-03   
...                                  ...           ...           ...   
ssp585 volcanic             1.999959e-01  1.999964e-01  1.730279e-01   
       solar                1.298828e-02 -2.142661e-02 -4.590393e-02   
       total_anthropogenic  1.125160e-02  5.608203e-03  1.247098e-02   
       total_natural        2.129842e-01  1.785697e-01  1.271240e-01   
       total                2.242358e-01  1.841780e-01  1.395949e-01   

year                                1756          1757          1758  \
ssp119 co2                  8.491360e-03  9.905408e-03  1.131912e-02   
       ch4                  3.045094e-03  3.551540e-03  4.057682e-03   
       n2o                  4.437396e-04  5.176679e-04  5.915882e-04   
       other_wmghg          4.774806e-09  5.570607e-09  6.366408e-09   
       o3                   1.800000e-03  2.100000e-03  2.400000e-03   
...                                  ...           ...           ...   
ssp585 volcanic            -7.767258e-02  6.635119e-02  1.532681e-01   
       solar               -5.348581e-02 -3.434389e-02  1.701324e-02   
       total_anthropogenic  1.692192e-02  1.047942e-02  1.587148e-02   
       total_natural       -1.311584e-01  3.200730e-02  1.702813e-01   
       total               -1.142365e-01  4.248672e-02  1.861528e-01   

year                                1759  ...       2491       2492  \
ssp119 co2                  1.273250e-02  ...   1.056188   1.055825   
       ch4                  4.563520e-03  ...   0.086943   0.086946   
       n2o                  6.655003e-04  ...   0.298153   0.298128   
       other_wmghg          7.162209e-09  ...   0.024382   0.024352   
       o3                   2.700000e-03  ...   0.031763   0.031767   
...                                  ...  ...        ...        ...   
ssp585 volcanic             1.842165e-01  ...   0.000000   0.000000   
       solar                6.751233e-02  ...   0.000000   0.000000   
       total_anthropogenic  1.649583e-02  ...  12.209823  12.207531   
       total_natural        2.517288e-01  ...   0.000000   0.000000   
       total                2.682247e-01  ...  12.209823  12.207531   

year                             2493       2494       2495       2496  \
ssp119 co2                   1.055462   1.055099   1.054735   1.054389   
       ch4                   0.086948   0.086951   0.086953   0.086955   
       n2o                   0.298106   0.298081   0.298059   0.298034   
       other_wmghg           0.024322   0.024292   0.024263   0.024233   
       o3                    0.031771   0.031775   0.031780   0.031784   
...                               ...        ...        ...        ...   
ssp585 volcanic              0.000000   0.000000   0.000000   0.000000   
       s

In [9]:
path = repo_ch7.retrieve('data_output/solar_erf.csv')
d_solar_erf = pd.read_csv(path, index_col='year').squeeze()

[2024-06-24 11:24:21 mce.util] INFO:Use local file datain/IPCC-WG1/Chapter-7/data_output/solar_erf.csv retrieved from https://github.com/IPCC-WG1/Chapter-7/raw/main/data_output/solar_erf.csv on 2024-06-21


In [10]:
d_solar_erf

year
-6755   -0.068556
-6754   -0.071130
-6753   -0.066856
-6752   -0.065373
-6751   -0.062737
           ...   
 2295   -0.004252
 2296    0.019050
 2297    0.044084
 2298    0.044732
 2299    0.025649
Name: solar_erf, Length: 9055, dtype: float64

In [11]:
fn = 'rcmip-emissions-annual-means-v5-1-0.csv'
path = retrieve_url(
    f'./datain/rcmip/{fn}',
    f'https://rcmip-protocols-au.s3-ap-southeast-2.amazonaws.com/v5.1.0/{fn}',
)
df = pd.read_csv(path, index_col=list(range(7))).rename(columns=int)

[2024-06-24 11:24:33 mce.util] INFO:Use local file datain/rcmip/rcmip-emissions-annual-means-v5-1-0.csv retrieved from https://rcmip-protocols-au.s3-ap-southeast-2.amazonaws.com/v5.1.0/rcmip-emissions-annual-means-v5-1-0.csv on 2024-05-25


In [12]:
species = ['BC', 'OC', 'Sulfur', 'NH3', 'NOx', 'VOC', 'CO']
df = (
    dffilter(
        df,
        Scenario=scenarios,
        Region='World',
        Variable=[f'Emissions|{x}' for x in species],
    )
    .interpolate(axis=1, pad=True)
    .rename(lambda x: x.split('|')[-1], level='Variable')
)

In [13]:
[k for k, _ in df.groupby(['Variable', 'Unit'])]

[('BC', 'Mt BC/yr'),
 ('CO', 'Mt CO/yr'),
 ('NH3', 'Mt NH3/yr'),
 ('NOx', 'Mt NOx/yr'),
 ('OC', 'Mt OC/yr'),
 ('Sulfur', 'Mt SO2/yr'),
 ('VOC', 'Mt VOC/yr')]

In [14]:
df_emis = (
    df
    .droplevel([x for x in df.index.names if x not in ['Scenario', 'Variable']])
    .rename({'Sulfur': 'SO2'}, level='Variable')
)
df_emis

1750        1751        1752        1753        1754  \
Scenario Variable                                                               
ssp370   BC          2.097771    2.072926    2.064312    2.071951    2.099350   
         CO        348.527359  343.926367  341.809034  343.675456  349.815625   
         NH3         6.927690    6.860142    6.819482    6.881439    7.015189   
         NOx        12.735212   12.592303   12.594270   12.539820   12.636837   
         OC         15.447668   15.188717   15.034476   15.182545   15.558773   
         SO2         2.440048    2.408379    2.397116    2.410693    2.462065   
         VOC        60.021826   59.088126   58.631997   59.026349   60.269945   
ssp119   BC          2.097771    2.072926    2.064312    2.071951    2.099350   
         CO        348.527359  343.926367  341.809034  343.675456  349.815625   
         NH3         6.927690    6.860142    6.819482    6.881439    7.015189   
         NOx        12.735212   12.592303   12.594270   12.539820   12.636837   
         OC         15.447668   15.188717   15.034476   15.182545   15.558773   
         SO2         2.440048    2.408379    2.397116    2.410693    2.462065   
         VOC        60.021826   59.088126   58.631997   59.026349   60.269945   
ssp126   BC          2.097771    2.072926    2.064312    2.071951    2.099350   
         CO        348.527359  343.926367  341.809034  343.675456  349.815625   
         NH3         6.927690    6.860142    6.819482    6.881439    7.015189   
         NOx        12.735212   12.592303   12.594270   12.539820   12.636837   
         OC         15.447668   15.188717   15.034476   15.182545   15.558773   
         SO2         2.440048    2.408379    2.397116    2.410693    2.462065   
         VOC        60.021826   59.088126   58.631997   59.026349   60.269945   
ssp245   BC          2.097771    2.072926    2.064312    2.071951    2.099350   
         CO        348.527359  343.926367  341.809034  343.675456  349.815625   
         NH3         6.927690    6.860142    6.819482    6.881439    7.015189   
         NOx        12.735212   12.592303   12.594270   12.539820   12.636837   
         OC         15.447668   15.188717   15.034476   15.182545   15.558773   
         SO2         2.440048    2.408379    2.397116    2.410693    2.462065   
         VOC        60.021826   59.088126   58.631997   59.026349   60.269945   
ssp585   BC          2.097771    2.072926    2.064312    2.071951    2.099350   
         CO        348.527359  343.926367  341.809034  343.675456  349.815625   
         NH3         6.927690    6.860142    6.819482    6.881439    7.015189   
         NOx        12.735212   12.592303   12.594270   12.539820   12.636837   
         OC         15.447668   15.188717   15.034476   15.182545   15.558773   
         SO2         2.440048    2.408379    2.397116    2.410693    2.462065   
         VOC        60.021826   59.088126   58.631997   59.026349   60.269945   

                         1755        1756        1757        1758        1759  \
Scenario Variable                                                               
ssp370   BC          2.099173    2.068915    2.147924    2.120611    2.117267   
         CO        348.144668  343.323152  355.941892  351.839749  351.507763   
         NH3         6.943989    6.922712    7.047674    6.995460    7.045958   
         NOx        12.765991   12.416163   13.133152   12.979345   12.752546   
         OC         15.358183   15.195777   15.704819   15.514612   15.574013   
         SO2         2.442355    2.419184    2.507465    2.479744    2.473528   
         VOC        59.824934   58.946829   61.263693   60.434612   60.445263   
ssp119   BC          2.099173    2.068915    2.147924    2.120611    2.117267   
         CO        348.144668  343.323152  355.941892  351.839749  351.507763   
         NH3         6.943989    6.922712    7.047674    6.995460    7.045958   
         NOx        12.765991   12.416163   13.133152  

In [15]:
# emissions_ceds_update
path = repo_ch7.retrieve('data_output/emissions_ceds_update_plus_bb.csv')
df_emis_ceds_update = pd.read_csv(path, index_col=0)

[2024-06-24 11:25:05 mce.util] INFO:Use local file datain/IPCC-WG1/Chapter-7/data_output/emissions_ceds_update_plus_bb.csv retrieved from https://github.com/IPCC-WG1/Chapter-7/raw/main/data_output/emissions_ceds_update_plus_bb.csv on 2024-06-13


In [16]:
# Merge data
df1 = pd.concat({
    scenario: df_emis_ceds_update.loc[:2014].T
    for scenario in scenarios
})
df2 = df_emis_ceds_update.loc[2015:2019].T.mul(np.linspace(1., 0.2, 5))
df2 = (
    pd.concat({scenario: df2 for scenario in scenarios})
    .rename_axis(['Scenario', 'Variable'])
)
df_emis = pd.concat([
    df1,
    df2.add(df_emis.loc[:, 2015:2019].mul(np.linspace(0., 0.8, 5))),
    df_emis.loc[:, 2020:],
], axis=1)

In [17]:
df_emis

1750        1751        1752        1753        1754  \
ssp119 BC     2.097336    2.072491    2.063877    2.071515    2.098914   
       CO   348.420476  343.819148  341.701465  343.567551  349.707369   
       NH3    6.683996    6.615537    6.573965    6.635011    6.767852   
       NOx   12.730074   12.587160   12.589121   12.534665   12.631676   
       OC    15.447054   15.188102   15.033861   15.181930   15.558158   
       SO2    2.312083    2.281498    2.270899    2.284976    2.336934   
       VOC   60.599376   59.667759   59.213713   59.610148   60.855827   
ssp126 BC     2.097336    2.072491    2.063877    2.071515    2.098914   
       CO   348.420476  343.819148  341.701465  343.567551  349.707369   
       NH3    6.683996    6.615537    6.573965    6.635011    6.767852   
       NOx   12.730074   12.587160   12.589121   12.534665   12.631676   
       OC    15.447054   15.188102   15.033861   15.181930   15.558158   
       SO2    2.312083    2.281498    2.270899    2.284976    2.336934   
       VOC   60.599376   59.667759   59.213713   59.610148   60.855827   
ssp245 BC     2.097336    2.072491    2.063877    2.071515    2.098914   
       CO   348.420476  343.819148  341.701465  343.567551  349.707369   
       NH3    6.683996    6.615537    6.573965    6.635011    6.767852   
       NOx   12.730074   12.587160   12.589121   12.534665   12.631676   
       OC    15.447054   15.188102   15.033861   15.181930   15.558158   
       SO2    2.312083    2.281498    2.270899    2.284976    2.336934   
       VOC   60.599376   59.667759   59.213713   59.610148   60.855827   
ssp370 BC     2.097336    2.072491    2.063877    2.071515    2.098914   
       CO   348.420476  343.819148  341.701465  343.567551  349.707369   
       NH3    6.683996    6.615537    6.573965    6.635011    6.767852   
       NOx   12.730074   12.587160   12.589121   12.534665   12.631676   
       OC    15.447054   15.188102   15.033861   15.181930   15.558158   
       SO2    2.312083    2.281498    2.270899    2.284976    2.336934   
       VOC   60.599376   59.667759   59.213713   59.610148   60.855827   
ssp585 BC     2.097336    2.072491    2.063877    2.071515    2.098914   
       CO   348.420476  343.819148  341.701465  343.567551  349.707369   
       NH3    6.683996    6.615537    6.573965    6.635011    6.767852   
       NOx   12.730074   12.587160   12.589121   12.534665   12.631676   
       OC    15.447054   15.188102   15.033861   15.181930   15.558158   
       SO2    2.312083    2.281498    2.270899    2.284976    2.336934   
       VOC   60.599376   59.667759   59.213713   59.610148   60.855827   

                  1755        1756        1757        1758        1759  ...  \
ssp119 BC     2.098738    2.068451    2.147459    2.120146    2.116802  ...   
       CO   348.036062  343.211697  355.830082  351.727584  351.395242  ...   
       NH3    6.695742    6.673551    6.797605    6.744482    6.794073  ...   
       NOx   12.760824   12.410995   13.127978   12.974166   12.747361  ...   
       OC    15.357567   15.195122   15.704165   15.513957   15.573358  ...   
       SO2    2.317138    2.295185    2.384057    2.354322    2.349376  ...   
       VOC   60.412899   59.537017   61.855964   61.028967   61.041701  ...   
ssp126 BC     2.098738    2.068451    2.147459    2.120146    2.116802  ...   
       CO   348.036062  343.211697  355.830082  351.727584  351.395242  ...   
       NH3    6.695742    6.673551    6.797605    6.744482    6.794073  ...   
       NOx   12.760824   12.410995   13.127978   12.974166   12.747361  ...   
       OC    15.357567   15.195122   15.704165   15.513957   15.573358  ...   
       SO2    2.317138    2.295185    2.384057    2.354322    2.349376  ...   
       VOC   60.412899   59.537017   61.855964   61.028967   61.041701  ...   
ssp245 BC     2.098738    2.068451    2.147459    2.120146    2.116802  ...   
       CO   348.036062  343.211697  355.830082  351.727584  351.395242  ...   
       NH3

## Forcing uncertainties

Same as that used in 050_unconstrained runs.ipynb

In [18]:
nsize = 50000

ncf = Dataset('./dataout/fair_samples_forcing.nc')

df_erf_scale = pd.DataFrame({
    k: ncf.variables[f'scale_normals__{k}'][:nsize]
    for k in [
        'co2', 'ch4', 'n2o', 'other_wmghg', 'o3', 'h2o_stratospheric',
        'contrails', 'bc_on_snow', 'land_use', 'volcanic', 'solar',
    ]
})

trend_solar = ncf.variables['trend_solar'][:nsize].filled()

aer_coeff = {
    k: ncf.variables[k][:nsize].filled()
    for k in ['beta_so2', 'beta_bc', 'beta_oc', 'beta_nh3', 'beta']
}
aer_coeff['aci_coeffs'] = np.array([
    ncf.variables['aci_coeffs__0'][:nsize].filled(),
    ncf.variables['aci_coeffs__1'][:nsize].filled(),
]).T

ncf.close()

## TLM parameter ensemble

Same as that used in 050_unconstrained runs.ipynb

In [19]:
ncf = Dataset('./dataout/parms_sample.nc')

method_order = [x for x in ncf.variables['Method'][:]]
names = ['q4x', 'q2x', 'lamg', 'cmix', 'cdeep', 'gamma_2l', 'eff', 'co2_beta']

df = [ncf.variables[k][:].filled(np.nan) for k in names]
df_parms_sample = pd.DataFrame(
    np.hstack(df),
    columns=pd.MultiIndex.from_product([names, method_order]),
)

ncf.close()

In [20]:
len(df_parms_sample) == nsize

True

## CO2 forcing scales

Same as that used in 050_unconstrained runs.ipynb

In [21]:
obj_rf = RfAll()

# Reference values of CO2 in 2019 and 4xCO2 forcing
cco2_pi = obj_rf.parms_ar6_ghg.C0_1750
obj_rf.parms.update(ccref=cco2_pi)

cco2_2019 = 409.85
cn2o_2019 = 332.091
erf_co2_2019 = obj_rf.c2erf_ar6('CO2', cco2_2019, cn2o=cn2o_2019)
q4x_ref = obj_rf.c2erf_ar6('CO2', cco2_pi*4.)
cco2_pi, cco2_2019, erf_co2_2019, cco2_pi*4., q4x_ref

(278.3, 409.85, 2.156277925173476, 1113.2, 8.259783657536742)

In [22]:
df_scale_co2 = pd.concat([
    df_parms_sample.loc[:, ('q4x', method)].div(q4x_ref).rename(method)
    for method in ['ar6', 's21']
], axis=1)

For MCE-2l, use equivalent CO2 concentrations, converted from well-mixed GHG ERF with the AR6 formula of CO2 ERF. Here, define an interpolation object to get the equivalent CO2 concentrations.

In [23]:
# Nominal range of well-mixed GHG ERF across the SSP scenarios
xlim = (-1., 12.)

# y-value: equivalent CO2 concentrations
alpha = erf_co2_2019 / np.log(cco2_2019/cco2_pi)
yp = cco2_pi * np.exp(np.linspace(*xlim, **{'num': 500}) / alpha)

# x-value: well-mixed GHG ERF
xp = obj_rf.c2erf_ar6('CO2', yp)

interp_cco2 = interp1d(xp, yp)

## Results from constraining

In [28]:
df_members = pd.read_csv(
    './dataout/constraining_results.csv',
    header=[0, 1], index_col=0,
)
df_members

Constraining ar6_chapter7                       mh_sampler                  \
Sampling              ar6 ar6_orig mce-2l   s21        ar6 ar6_orig mce-2l   
Sequence                                                                     
0                       2        7     11     7       7796    24312   9649   
1                      13       11     15    12      10266    23899  34909   
2                      16       15     22    13      12618    46289  17814   
3                      22       16     37    15      25845    16155  45096   
4                      24       24     38    16      17894    13573  30940   
...                   ...      ...    ...   ...        ...      ...    ...   
595                  3707     3460   4942  3444      46701    11904   8243   
596                  3724     3477   4957  3451      44151    32570  46505   
597                  3727     3481   4961  3458      30470    11207  41679   
598                  3736     3483   4983  3462       6294    32012   8265   
599                  3741     3489   4989  3471      44240    30559  20166   

Constraining         
Sampling        s21  
Sequence             
0             22342  
1             24905  
2             38955  
3              6674  
4             48938  
...             ...  
595           36618  
596            4038  
597           47393  
598           37515  
599            4269  

[600 rows x 8 columns]

In [31]:
mi = [({'ar6_orig': 'ar6_chapter7'}.get(k, 'mh_sampler'), k) for k in method_order]
mi

[('mh_sampler', 'ar6'),
 ('ar6_chapter7', 'ar6_orig'),
 ('mh_sampler', 'mce-2l'),
 ('mh_sampler', 's21')]

In [35]:
df_members = df_members[mi].droplevel(0, axis=1)
df_members

Sampling,ar6,ar6_orig,mce-2l,s21
Sequence,,,,
0,7796,7,9649,22342
1,10266,11,34909,24905
2,12618,15,17814,38955
3,25845,16,45096,6674
4,17894,24,30940,48938
...,...,...,...,...
595,46701,3460,8243,36618
596,44151,3477,46505,4038
597,30470,3481,41679,47393


## Ensemble member loop

In [36]:
cats = [
    'co2', 'ch4', 'n2o', 'other_wmghg', 'o3',
    'h2o_stratospheric', 'contrails', 'bc_on_snow', 'land_use',
    'volcanic', 'solar',
]
cats_aerosol = [
    'aerosol-radiation_interactions',
    'aerosol-cloud_interactions',
]
cats_nonco2 = cats[1:] + cats_aerosol

In [37]:
df_emis_a = df_emis.sub(df_emis.loc[:, 1750], axis=0)

In [38]:
with open('./src/variables.yml') as f1:
    var_atts = yaml.safe_load(f1)

In [39]:
variable_order = ['tg', 'ohc', 'erf', 'erf_co2']

year_range = (1850, 2014)
df = pd.DataFrame(
    0.,
    index=pd.Index([0], name='Member'),
    columns=pd.MultiIndex.from_product([
        variable_order,
        method_order,
        df_erf.loc[:, slice(*year_range)].columns,
    ], names=['Variable', 'Method', 'Year'])
)
path_out = 'dataout/constrained_run_hist.nc'
df2nc(path_out, df, var_atts)
ncf_hist = Dataset(path_out, 'r+')


year_range = (2015, 2100)

df = pd.DataFrame(
    0.,
    index=pd.Index([0], name='Member'),
    columns=pd.MultiIndex.from_product([
        variable_order,
        method_order,
        scenarios,
        df_erf.loc[:, slice(*year_range)].columns,
    ], names=['Variable', 'Method', 'Scenario', 'Year'])
)

path_out = 'dataout/constrained_run_ssps.nc'
df2nc(path_out, df, var_atts)
ncf_ssps = Dataset(path_out, 'r+')

[2024-06-24 11:36:05 __name__] INFO:Old dataout/constrained_run_hist.nc is deleted
[2024-06-24 11:36:05 __name__] INFO:dataout/constrained_run_hist.nc is created
[2024-06-24 11:36:05 __name__] INFO:Old dataout/constrained_run_ssps.nc is deleted
[2024-06-24 11:36:05 __name__] INFO:dataout/constrained_run_ssps.nc is created


In [40]:
driver_mce = DriverMCE()

In [51]:
year_base = 1850

for i, d_member in tqdm(df_members.iterrows(), total=len(df_members)):
    for jm, method in enumerate(method_order):
        im = d_member.loc[method]
        p1 = df_parms_sample.loc[im].unstack()[method].dropna()
        driver_mce.calib(p1)

        for jscen, (scenario, df) in enumerate(df_erf.loc[:, :2100].groupby(level=0)):
            dfin = df.loc[(scenario, cats), :].mul(pd.concat({scenario: df_erf_scale.loc[im]}), axis=0)

            d1 = d_solar_erf.loc[1750:2100] * df_erf_scale.loc[im, 'solar']
            d1.loc[:2019] += np.linspace(0., trend_solar[im], len(d1.loc[:2019]))
            d1.loc[2020:] += trend_solar[im]
            dfin.loc[(scenario, 'solar'), :2100] = d1

            d1 = -aer_coeff['beta'][im] * np.log(
                1.
                + df_emis.loc[(scenario, 'SO2'), :].mul(32./64.) / aer_coeff['aci_coeffs'][im, 0]
                + df_emis.loc[(scenario, ['BC', 'OC']), :].sum() / aer_coeff['aci_coeffs'][im, 1]
            )
            dfa = pd.concat({
                (scenario, 'aerosol-radiation_interactions'):
                df_emis_a.loc[(scenario, 'SO2'), :] * aer_coeff['beta_so2'][im] * 32./64.
                + df_emis_a.loc[(scenario, 'BC'), :] * aer_coeff['beta_bc'][im]
                + df_emis_a.loc[(scenario, 'OC'), :] * aer_coeff['beta_oc'][im]
                + df_emis_a.loc[(scenario, 'NH3'), :] * aer_coeff['beta_nh3'][im],
                (scenario, 'aerosol-cloud_interactions'): d1.sub(d1.loc[1750]),
            }).unstack().loc[:, :2100]

            dfin = dfin.append(dfa).droplevel(level=0)

            # same configuration as in AR6 Chapter 4
            d_volcanic = dfin.loc['volcanic']
            d_total = dfin.sum().sub(d_volcanic)
            d_total = d_total.sub(d_total.loc[year_base]).add(d_volcanic)
            d_co2 = dfin.loc['co2'].sub(dfin.loc['co2', year_base])

            if method == 'ar6_orig':
                din_co2 = d_co2
            elif method in ['ar6', 's21']:
                # Perturbed CO2 forcing is further scaled
                din_co2 = d_co2 * df_scale_co2.loc[im, method]
            elif method == 'mce-2l':
                # Convert CO2 forcing to equivalent CO2 concentrations
                # and calculate forcing with the MCE CO2 scheme
                obj_rf.parms.update(
                    alpha=p1['q4x'] / (np.log(4.) * p1['co2_beta']),
                    beta=p1['co2_beta'],
                )
                din_co2 = pd.Series(obj_rf.c2erf(interp_cco2(d_co2)), index=d_co2.index)
            else:
                raise ValueError(f'unexpected method {method}')

            din = din_co2 + d_total.sub(d_co2)

            if jscen == 0:
                # historical period
                slc = slice(1850, 2014)
                dfout = driver_mce.run(din.loc[slc])
                dfout['erf'] = din.loc[slc]
                dfout['erf_co2'] = din_co2.loc[slc]
                tkjlast = driver_mce.climate.tkjlast
                qlast = din.loc[2014]

                ncf_hist.variables['Member'][i] = i
                ncf_ssps.variables['Member'][i] = i
                for vn, d1 in dfout.items():
                    if vn not in variable_order:
                        continue
                    ncf_hist.variables[vn][i, jm, :] = d1.values

            slc = slice(2015, 2100)
            dfout = driver_mce.run(din.loc[slc], tkjlast=tkjlast, qlast=qlast)
            dfout['erf'] = din.loc[slc]
            dfout['erf_co2'] = din_co2.loc[slc]

            for vn, d1 in dfout.items():
                if vn not in variable_order:
                    continue
                ncf_ssps.variables[vn][i, jm, jscen, :] = d1.values

  0%|          | 0/600 [00:00<?, ?it/s]

In [52]:
ncf_hist.close()
ncf_ssps.close()